In [216]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pandas as pd
import geopandas as gpd
import rasterio
import osmnx as ox
import json
import geojson
from math import ceil, floor
import cv2
import random

from shapely.geometry import LineString, CAP_STYLE
from shapely.ops import transform
from shapely import wkt
import shapely.geometry as gm

import skimage
import io


from skimage import measure, morphology
from rasterio import features as raster_features
from rasterio.plot import show
from rasterio.enums import MergeAlg

from bng_latlon import OSGB36toWGS84
from geojson import Polygon, Feature, FeatureCollection

In [217]:
min_height = 5.2
width = 5
corridorHeight = 10

DTM = 'DTM.tif'
DSM = 'DSM.tif'

In [218]:
voltages = {
    'ESO': {
        'Voltages': ['275kV', '400kV'],
        'Company': 'National Grid',
        'Priority': 0.7
    },
    'DNO': {
        'Voltages': ['132kV','33kV', '11kV'],
        'Company': 'Northern Power',
        'Priority': 0.5
    }
}

#https://www.hse.gov.uk/pubns/ais8.pdf

In [219]:
raster = rasterio.open(DTM)

In [220]:
## bbox - top left, top right, bottom left, bottom right
bbox = [[raster.bounds.left, raster.bounds.top], [raster.bounds.right, raster.bounds.top], 
        [raster.bounds.left, raster.bounds.bottom], [raster.bounds.right, raster.bounds.bottom]]

bbox_wsg84 = []
for x in bbox:
    x, y = OSGB36toWGS84(x[0], x[1])
    bbox_wsg84.append([x, y])

left = bbox_wsg84[2][1]
right = bbox_wsg84[1][1]
bottom = bbox_wsg84[3][0]
top = bbox_wsg84[0][0]

In [221]:
with open('DNO-Network.geojson') as f:
    dno = geojson.load(f)
with open('ESO-Cabel.geojson') as f:
    eso_cable = geojson.load(f)
with open('ESO-OHL.geojson') as f:
    eso_ohl = geojson.load(f)

for x in dno['features']:
    x['properties']['network'] = 'DNO'
    x['properties']['stroke'] = '#40bf40'
    x['properties']['stroke-opacity'] = 0.5
    x['properties']['priority'] = voltages[x['properties']['network']]['Priority']

for x in eso_cable['features']:
    x['properties']['network'] = 'ESO'
    x['properties']['stroke'] = '#6c936c'
    x['properties']['stroke-opacity'] = 0.3
    x['properties']['priority'] = 0.1

for x in eso_ohl['features']:
    x['properties']['network'] = 'ESO'
    x['properties']['stroke'] = '#00ff00'
    x['properties']['stroke-opacity'] = 0.7
    x['properties']['priority'] = voltages[x['properties']['network']]['Priority']




fs = dno['features'] + eso_cable['features'] + eso_ohl['features']

feature_collection = FeatureCollection(fs)
with open('NE-power-network.geojson', 'w') as f:
   json.dump(feature_collection, f)

In [222]:
edges = gpd.read_file('NE-power-network.geojson', driver='GeoJSON')
edges = edges.cx[left:right, bottom:top]
edges.to_file('NE-power-network-trimmed.geojson', driver='GeoJSON')


In [223]:
edges_osgb = edges.to_crs(raster.crs)
buffer_network = [(shapes.buffer(width, cap_style=CAP_STYLE.round)) for shapes in edges_osgb.geometry]


In [224]:
lidar_dtm = skimage.img_as_float(skimage.io.imread(DTM))
lidar_dsm = skimage.img_as_float(skimage.io.imread(DSM))
CHM = (lidar_dsm - lidar_dtm)
CHM_threshold = np.multiply(CHM,100)

In [225]:
# # CHM[5340:5360,11260:11280]
# np.multiply(CHM[5350:5355,11265:11270],250)

In [226]:
# plt.imshow(CHM[5350:5355,11265:11270])

In [227]:
# plt.imshow(CHM[5240:5460,11160:11380])

In [228]:
trigger = (CHM_threshold > min_height)
CHM_threshold[trigger == False] = 0


In [229]:
for ind, x in enumerate(buffer_network):
    edgeLength = edges_osgb.loc[edges_osgb.index[ind], 'Length']
    edgeVolume = edgeLength * width * corridorHeight
    buffer_shape = (ceil(x.bounds[2] - x.bounds[0]), ceil(x.bounds[3] - x.bounds[1]))

    data = [x]
    rasterized = raster_features.rasterize(data,
                                out_shape = raster.shape,
                                fill = 0,
                                out = None,
                                transform = raster.transform,
                                all_touched = False,
                                default_value = 1,
                                dtype = None)


    raster_bounds = (x.bounds[0] - raster.bounds[0]), (raster.bounds[3] - x.bounds[3]), (x.bounds[0] - raster.bounds[0] + (x.bounds[2] - x.bounds[0])), (raster.bounds[3] - x.bounds[3] + (x.bounds[3] - x.bounds[1]))
    raster_bounds_clip = floor(x.bounds[0] - raster.bounds[0]), floor(raster.bounds[3] - x.bounds[3]), ceil(x.bounds[0] - raster.bounds[0] + (x.bounds[2] - x.bounds[0])), ceil(raster.bounds[3] - x.bounds[3] + (x.bounds[3] - x.bounds[1]))

    rasterized_mask = rasterized > 0

    rasterized_clip = rasterized[raster_bounds_clip[1]:raster_bounds_clip[3], raster_bounds_clip[0]:raster_bounds_clip[2]]
    rasterized_mask = rasterized_clip > 0
    CHM_clip = CHM_threshold[raster_bounds_clip[1]:raster_bounds_clip[3], raster_bounds_clip[0]:raster_bounds_clip[2]]
    CHM_clip[rasterized_mask == False] = 0
    total_over = np.sum(CHM_clip)
    if total_over > 0 and x.area > 0:
        volume_over = np.subtract(CHM_clip,[min_height], where=CHM_clip>min_height)
        percent_covered = (np.sum(volume_over))/(x.area*(min_height*2))
        print(ind, total_over, percent_covered, round(percent_covered*100,2))

        edges_osgb.at[edges_osgb.index[ind], 'AtRisk'] = True
        edges_osgb.at[edges_osgb.index[ind], 'PercentExposed'] = f"{round(percent_covered*100,2)}%"
        edges_osgb.at[edges_osgb.index[ind], 'stroke'] = '#ff0000'
    else:
        edges_osgb.at[edges_osgb.index[ind], 'AtRisk'] = False
        edges_osgb.at[edges_osgb.index[ind], 'PercentExposed'] = f"0%"
        

        


18 20.7843137254902 1.0973510416186825e-05 0.0
20 36.47058823529413 1.941101544520839e-05 0.0
37 10.98039215686275 1.2285019225383275e-06 0.0
62 5.882352941176472 8.994356120871854e-07 0.0
72 5.490196078431375 1.0133270865124783e-06 0.0
90 1309.8039215686276 0.6956715414708884 69.57
93 665.4901960784315 0.032215575292666304 3.22
98 1532.549019607843 0.8296041145779762 82.96
101 1258.8235294117649 0.25281406721407756 25.28
105 665.4901960784315 0.032215575292666325 3.22
109 1532.549019607843 0.829604114577976 82.96
110 1309.8039215686276 1.2201660804628731 122.02
114 1258.8235294117649 0.758004470475452 75.8
115 5.490196078431375 9.186518524889565e-07 0.0
116 5.490196078431375 9.186518524889662e-07 0.0
133 5.882352941176472 8.994356120871854e-07 0.0
265 11.764705882352942 2.3004289554086592e-06 0.0
270 143.921568627451 0.00011474490996492155 0.01
271 1631.3725490196075 0.0015662341334095533 0.16


In [230]:
edges_output = edges_osgb.to_crs(edges.crs)

In [231]:
edges_output.to_file('NE-power-network-risk.geojson', driver='GeoJSON')